In [56]:
import pandas as pd
import os
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from torch.utils import data as data_torch
import torch
from torch.utils import data as data_torch
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [57]:
path="/home/data/meme_challenge_mod_data"
title="simple_"

image_features=pd.read_csv(os.path.join(path,title+"image_features.csv"),sep=",",index_col=0)
text_features=pd.read_csv(os.path.join(path,title+"text_features.csv"),sep=",",index_col=0)
desc_features=pd.read_csv(os.path.join(path,title+"description.csv"),sep=",",index_col=0)
''' 
results.to_csv()

results=pd.DataFrame(desc)
results.to_csv(os.path.join(path,title+"description.csv"),sep=",")

results=pd.DataFrame(img_features)
results.to_csv(os.path.join(path,title+"image_features.csv"),sep=",")
'''

' \nresults.to_csv()\n\nresults=pd.DataFrame(desc)\nresults.to_csv(os.path.join(path,title+"description.csv"),sep=",")\n\nresults=pd.DataFrame(img_features)\nresults.to_csv(os.path.join(path,title+"image_features.csv"),sep=",")\n'

In [58]:
from nn_simple import SimpleModel

In [59]:
class Dataset_Handler_Generic(data_torch.Dataset):
    def __init__(self, data, label):
        
            self.data=data #how is the data bbroken  down
            self.label=label
          
    def __len__(self):
            'Denotes the total number of samples'
            return len(self.data)
    def __getitem__(self, index):
            return  self.data[index],self.label[index]

In [60]:
input_features=pd.concat((image_features,text_features),axis=1).values


In [63]:
output_features=desc_features['1'].values.reshape((len(desc_features['1']),1))

In [64]:
size=len(input_features)

train_size=int(size*.80)

handler_tr:data_torch.Dataset=Dataset_Handler_Generic(input_features[:train_size],output_features[:train_size])
handler_vl:data_torch.Dataset=Dataset_Handler_Generic(input_features[train_size+1:],output_features[train_size+1:])    


In [65]:

loader_tr=torch.utils.data.DataLoader(handler_tr, batch_size=10, shuffle=True,num_workers=0)
loader_vl=torch.utils.data.DataLoader(handler_vl, batch_size=10, shuffle=True,num_workers=0)


(8496, 1768)

In [164]:
lossType=nn.SmoothL1Loss(reduction="sum")
s=SimpleModel(input_features.shape[1],1,[],lossType,lr=.001)

In [168]:

optimizer=torch.optim.Adam(s.parameters(),lr=.0001,weight_decay=.001)


In [169]:
epochs=5
norm_tr_size:int=len(input_features[:train_size])
norm_vl_size:int=len(input_features[train_size+1:])
    
initial_tr_loss=s.train_model(1,loader_tr,optimizer,is_training=False)[0]/norm_tr_size
intial_vl_loss=s.train_model(1,loader_vl,optimizer,is_training=False)[0]/norm_vl_size
tr_loss_ary=[initial_tr_loss]
vl_loss_ary=[intial_vl_loss]
print(initial_tr_loss,intial_vl_loss)
for i in range(epochs):
    tr_loss=s.train_model(1,loader_tr,optimizer,is_training=True)[0]/norm_tr_size
    vl_loss=s.train_model(1,loader_vl,optimizer,is_training=False)[0]/norm_vl_size
    tr_loss_ary.append(tr_loss)
    vl_loss_ary.append(vl_loss)
    print(tr_loss,vl_loss)

0.10082503976086576 0.12227125431664766
0.08744562338772348 0.10457203793764255
0.08564200578120401 0.1072271301018062
0.0854164738736481 0.1066577859735966
0.08499762505345516 0.10307447625441436
0.08469752356050433 0.10283715223971363


In [170]:
results=s(torch.tensor(input_features[:train_size]).float()).detach().numpy()
predicted_label=np.apply_along_axis(lambda d: 1 if d >.5 else 0,1,results)

In [171]:

confusion_matrix(output_features[:train_size],predicted_label)



array([[3565,  882],
       [ 800, 1549]])

In [172]:
accuracy_score(output_features[:train_size],predicted_label)

0.7525014714537963

In [173]:
from sklearn.metrics import roc_auc_score

In [174]:
results_vl=s(torch.tensor(input_features[train_size+1:]).float()).detach().numpy()
predicted_label_vl=np.apply_along_axis(lambda d: 1 if d >.5 else 0,1,results_vl)

In [175]:
accuracy_score(output_features[train_size+1:],predicted_label_vl)

0.6909947027663331

In [176]:
pd.DataFrame(output_features)[0].value_counts()

0    5446
1    3050
Name: 0, dtype: int64

In [177]:
roc_auc_score(output_features[train_size+1:],predicted_label_vl)

0.6798747280581132

In [178]:
roc_auc_score(output_features[:train_size],predicted_label)

0.730546793831095